In [1]:
from __future__ import print_function
from pipython import GCSDevice,pitools
import serial.tools.list_ports
from pipython.gcserror import GCSError
import sys
import os
import numpy as np;
import libtiepie
import time

start_time = time.time()


In [2]:
class tiepie:
    
    def __init__(self):

        libtiepie.network.auto_detect_enabled = True
        libtiepie.device_list.update()

        # Try to open an oscilloscope with block measurement support and a generator in the same device:
        self.scp = None
        self.gen = None
        
        for item in libtiepie.device_list:
            if (item.can_open(libtiepie.DEVICETYPE_OSCILLOSCOPE)):
                  if self.scp.measure_modes & libtiepie.MM_BLOCK:
                    self.scp = item.open_oscilloscope()
                
            if (item.can_open(libtiepie.DEVICETYPE_GENERATOR)):
                if self.scp.measure_modes & libtiepie.MM_BLOCK:
                    self.gen = item.open_generator()
                    break
                else:
                    self.scp = None

        if self.scp and self.gen:
            try:
                self.scp.measure_mode = libtiepie.MM_BLOCK
                self.scp.sample_frequency = 1e6  # 1 MHz
                self.scp.record_length = 10000  # 10000 samples
                self.scp.pre_sample_ratio = 0  # 0 %

                # For all channels:
                for ch in self.scp.channels:
                    ch.enabled = True
                    ch.range = 8  # 8 V
                    ch.coupling = libtiepie.CK_DCV  # DC Volt

                # Set trigger timeout:
                self.scp.trigger_time_out = 1  # 1 s

                # Disable all channel trigger sources:
                for ch in self.scp.channels:
                    ch.trigger.enabled = False

                # Locate trigger input:
                self.trigger_input = self.scp.trigger_inputs.get_by_id(libtiepie.TIID_GENERATOR_NEW_PERIOD)  # or TIID_GENERATOR_START or TIID_GENERATOR_STOP

                if self.trigger_input is None:
                    raise Exception('Unknown trigger input!')

                # Enable trigger input:
                self.trigger_input.enabled = True
                
                self.gen.signal_type = libtiepie.ST_TRIANGLE
                self.gen.frequency = 1e3  # 1 kHz
                self.gen.amplitude = 2  # 2 V
                self.gen.offset = 0  # 0 V
                self.gen.output_on = True
                
            except Exception as e:
                print('Exception: ' + e.message)
                sys.exit(1)
        else:
            print('No oscilloscope available with block measurement support or generator available in the same unit!')

    def take_reading(self):
        
        self.scp.start()
        self.gen.start()

        # Wait for measurement to complete:
        while not self.scp.is_data_ready:
            time.sleep(0.01)  # 10 ms delay, to save CPU time

        # Stop generator:
        self.gen.stop()
        self.gen.output_on = False

        # Get data:
        data = self.scp.get_data()

        # Output CSV data:
        csv_file = open('OscilloscopeGeneratorTrigger.csv', 'w')
        try:
            csv_file.write('Sample')
            for i in range(len(data)):
                csv_file.write(';Ch' + str(i + 1))
            csv_file.write(os.linesep)
            for i in range(len(data[0])):
                csv_file.write(str(i))
                for j in range(len(data)):
                    csv_file.write(';' + str(data[j][i]))
                csv_file.write(os.linesep)

            print()
            print('Data written to: ' + csv_file.name)

        finally:
            csv_file.close()
        # Close oscilloscope:
        del self.scp
        del self.gen

In [3]:
print(len(libtiepie.device_list))
x = libtiepie.device_list.update()
print(x)

0
None


In [86]:
del x

NameError: name 'x' is not defined

In [4]:
x = tiepie()

AttributeError: 'NoneType' object has no attribute 'measure_modes'